### Here we compare the different validation techniques on an imagenet model already trained to 93%

In [27]:
import argparse, os, shutil, time, warnings
from datetime import datetime
from pathlib import Path
import numpy as np
import sys

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [28]:
from torch.utils.data.sampler import Sampler
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from tqdm import tqdm
import os.path
import urllib.request
import pandas as pd

In [29]:
cudnn.benchmark = True
data = Path.home()/'data/imagenet'
workers = 7
valdir = os.path.join(data, 'validation')
batch_size = 128

## Create Image to Aspect ratio mapping

In [30]:
# Step 1: sort images by aspect ratio
def sort_ar(valdir):
    idx2ar_file = data/'sorted_idxar.p'
    if os.path.isfile(idx2ar_file): return pickle.load(open(idx2ar_file, 'rb'))
    print('Creating AR indexes. Please be patient this may take a couple minutes...')
    val_dataset = datasets.ImageFolder(valdir)
    sizes = [img[0].size for img in tqdm(val_dataset, total=len(val_dataset))]
    idx_ar = [(i, round(s[0]/s[1], 5)) for i,s in enumerate(sizes)]
    sorted_idxar = sorted(idx_ar, key=lambda x: x[1])
    pickle.dump(sorted_idxar, open(idx2ar_file, 'wb'))
    return sorted_idxar

# Step 2: chunk images by batch size. This way we can crop each image to the batch aspect ratio mean 
def chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

# Step 3: map image index to batch aspect ratio mean so our transform function knows where to crop
def map_idx2ar(idx_ar_sorted, batch_size):
    ar_chunks = list(chunks(idx_ar_sorted, batch_size))
    idx2ar = {}
    ar_means = []
    for chunk in ar_chunks:
        idxs, ars = list(zip(*chunk))
        mean = round(np.mean(ars), 5)
        ar_means.append(mean)
        for idx in idxs:
            idx2ar[idx] = mean
    return idx2ar, ar_means

In [31]:
idx_ar_sorted = sort_ar(valdir)

#### OR just download it

In [32]:
idx2ar_path = data/'sorted_idxar.p'
url = 'https://s3-us-west-2.amazonaws.com/ashaw-fastai-imagenet/sorted_idxar.p'
if not os.path.exists(idx2ar_path): urllib.request.urlretrieve(url, idx2ar_path)
idx_ar_sorted = sort_ar(valdir)

## Dataset Preparation

In [33]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform, target_transform)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            for tfm in self.transform:
                if isinstance(tfm, RectangularCropTfm): sample = tfm(sample, index)
                else: sample = tfm(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target

    
# Essentially a sequential sampler
class SequentialIndexSampler(Sampler):
    def __init__(self, indices): self.indices = indices
    def __len__(self): return len(self.indices)
    def __iter__(self): return iter(self.indices)
    


In [34]:
class RectangularCropTfm(object):
    def __init__(self, idx2ar, target_size):
        self.idx2ar, self.target_size = idx2ar, target_size
    def __call__(self, img, idx):
        target_ar = self.idx2ar[idx]
        if target_ar < 1: 
            w = int(self.target_size/target_ar)
            size = (w//8*8, self.target_size)
        else: 
            h = int(self.target_size*target_ar)
            size = (self.target_size, h//8*8)
        return torchvision.transforms.functional.center_crop(img, size)

## Validation Function with TTA

In [35]:
def validate(val_loader, model, criterion, aug_loader=None, num_augmentations=0):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    start_time = datetime.now()

    model.eval()
    end = time.time()

    val_iter = iter(val_loader)
    aug_iters = [iter(aug_loader) for i in range(num_augmentations)]
    prec5_arr = []
    for i in range(len(val_loader)):
        def get_output(dl_iter):
            input,target = next(dl_iter)
            input, target = input.cuda(), target.cuda()

            # compute output
            with torch.no_grad():
                output = model(Variable(input))
                loss = criterion(output, Variable(target))
            return output, loss, input, target
        
        output,loss,input,target = get_output(val_iter)
        for aug_iter in aug_iters:
            o,l,_,_ = get_output(aug_iter)
            output.add_(o)
            loss.add_(l)
        loss.div_(num_augmentations+1)
        
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        reduced_loss = loss.data
            
        losses.update(to_python_float(reduced_loss), input.size(0))
        top1.update(to_python_float(prec1), input.size(0))
        top5.update(to_python_float(prec5), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if (i != 0) and (i%50 == 0):
            prec5_arr.append(top5.val)
            output = ('Test: [{0}/{1}]\t' \
                    + 'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                    + 'Loss {loss.val:.4f} ({loss.avg:.4f})\t' \
                    + 'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t' \
                    + 'Prec@5 {top5.val:.3f} ({top5.avg:.3f})').format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    top1=top1, top5=top5)
            print(output)

    time_diff = datetime.now()-start_time
    print(f'~~{float(time_diff.total_seconds() / 3600.0)}\t{top5.avg:.3f}\n')
    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'.format(top1=top1, top5=top5))

    return prec5_arr

# item() is a recent addition, so this helps with backward compatibility.
def to_python_float(t):
    if hasattr(t, 'item'): return t.item()
    else: return t[0]
    
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self): self.reset()
    def reset(self): self.val = self.avg = self.sum = self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [42]:
import resnet
model = resnet.resnet50()
model = model.cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [40]:
model_path = data/'93_base_resnet50.p'
url = 'https://s3-us-west-2.amazonaws.com/ashaw-fastai-imagenet/93_base_resnet50.p'
if not os.path.exists(model_path): urllib.request.urlretrieve(url, model_path)

In [ ]:
model.load_state_dict(torch.load(model_path))

### Custom Data loaders

In [13]:
def tfm_wrapper_idx(fn):
    return lambda x,idx: (fn(x),idx)
def tfm_wrapper(fn):
    return lambda x,idx: fn(x)

In [14]:
class ValDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super().__init__(root, transform, target_transform)
    def __getitem__(self, index):
        path, target = self.imgs[index]
        sample = self.loader(path)
        if self.transform is not None:
            for tfm in self.transform:
                if isinstance(tfm, RectangularCropTfm): sample = tfm(sample, index)
                else: sample = tfm(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return sample, target


### Global dataset settings

In [15]:
val_bs = 128
target_size = 288

idx_sorted, _ = zip(*idx_ar_sorted)
idx2ar, ar_means = map_idx2ar(idx_ar_sorted, val_bs)
val_sampler_ar = SequentialIndexSampler(idx_sorted)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
tensor_tfm = [transforms.ToTensor(), normalize]

### Test validation with aspect ratio transforms

In [ ]:
val_dataset_ar = ValDataset(valdir, [RectangularCropTfm(idx2ar, target_size)] + tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_prec5 = validate(val_loader, model, criterion)

### Test Original Validation Technique (

In [19]:
val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

orig_prec5 = validate(val_loader, model, criterion)

Test: [10/391]	Time 0.099 (0.357)	Loss 1.2266 (1.0176)	Prec@1 74.219 (73.509)	Prec@5 88.281 (91.832)
Test: [20/391]	Time 0.099 (0.286)	Loss 0.8472 (1.0271)	Prec@1 77.344 (73.772)	Prec@5 95.312 (91.592)
Test: [30/391]	Time 0.099 (0.260)	Loss 0.9014 (1.0098)	Prec@1 75.781 (74.068)	Prec@5 94.531 (91.986)
Test: [40/391]	Time 0.099 (0.259)	Loss 1.1865 (0.9693)	Prec@1 69.531 (75.191)	Prec@5 89.062 (92.454)
Test: [50/391]	Time 0.099 (0.255)	Loss 1.2139 (1.0197)	Prec@1 70.312 (74.265)	Prec@5 91.406 (91.850)
Test: [60/391]	Time 0.668 (0.249)	Loss 1.4102 (1.0635)	Prec@1 62.500 (73.309)	Prec@5 88.281 (91.317)
Test: [70/391]	Time 0.099 (0.238)	Loss 0.9673 (1.0534)	Prec@1 78.125 (73.537)	Prec@5 92.188 (91.395)
Test: [80/391]	Time 0.099 (0.230)	Loss 0.9507 (1.0258)	Prec@1 75.781 (74.199)	Prec@5 90.625 (91.763)
Test: [90/391]	Time 0.099 (0.229)	Loss 0.8936 (1.0128)	Prec@1 76.562 (74.493)	Prec@5 95.312 (92.033)
Test: [100/391]	Time 0.099 (0.224)	Loss 0.8130 (0.9908)	Prec@1 78.906 (74.961)	Prec@5 94.53

### Test AR with size*1.14

In [21]:
val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

ar_rs_prec5 = validate(val_loader, model, criterion)

Test: [10/391]	Time 0.146 (1.121)	Loss 1.0566 (0.9035)	Prec@1 76.562 (76.207)	Prec@5 92.188 (93.821)
Test: [20/391]	Time 2.574 (0.854)	Loss 0.8447 (0.9367)	Prec@1 80.469 (76.153)	Prec@5 95.312 (93.787)
Test: [30/391]	Time 0.127 (0.881)	Loss 0.9351 (0.9399)	Prec@1 74.219 (76.008)	Prec@5 92.969 (93.674)
Test: [40/391]	Time 0.127 (0.697)	Loss 1.1084 (0.9104)	Prec@1 71.094 (76.963)	Prec@5 92.188 (93.921)
Test: [50/391]	Time 0.159 (0.604)	Loss 1.0420 (0.9545)	Prec@1 75.000 (76.011)	Prec@5 91.406 (93.290)
Test: [60/391]	Time 0.568 (0.544)	Loss 1.3164 (0.9920)	Prec@1 65.625 (75.102)	Prec@5 89.844 (92.777)
Test: [70/391]	Time 0.158 (0.519)	Loss 0.9653 (0.9864)	Prec@1 77.344 (75.165)	Prec@5 92.188 (92.749)
Test: [80/391]	Time 2.509 (0.546)	Loss 0.8687 (0.9627)	Prec@1 76.562 (75.714)	Prec@5 92.969 (93.036)
Test: [90/391]	Time 0.112 (0.555)	Loss 0.8076 (0.9521)	Prec@1 81.250 (75.893)	Prec@5 96.875 (93.192)
Test: [100/391]	Time 0.098 (0.584)	Loss 0.8130 (0.9349)	Prec@1 78.906 (76.214)	Prec@5 94.53

## Comparison

In [34]:
ar_batch_means = [np.array(c).mean() for c in chunks(ar_means, 10)]

In [38]:
d = {'DawnValidation': orig_prec5, 
     'BatchAspectRatioValidation': ar_rs_prec5, 
     'AR Mean': ar_batch_means[:-1],
     'Difference': np.array(ar_rs_prec5)-np.array(orig_prec5)}
df = pd.DataFrame(data=d);
df.to_csv('ar_tests.csv')
df

,DawnValidation,BatchAspectRatioValidation,AR Mean,Difference
0,88.28125,92.18750,0.636959,3.90625
1,95.31250,95.31250,0.667781,0.00000
2,94.53125,92.96875,0.717153,-1.56250
3,89.06250,92.18750,0.750000,3.12500
4,91.40625,91.40625,0.750000,0.00000
5,88.28125,89.84375,0.750000,1.56250
6,92.18750,92.18750,0.750127,0.00000
7,90.62500,92.96875,0.777394,2.34375
8,95.31250,96.87500,0.830958,1.56250
9,94.53125,94.53125,0.922669,0.00000


### Test TTA with original validation

In [39]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)


In [40]:
tta_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

Test: [10/391]	Time 0.693 (1.828)	Loss 1.2256 (1.0097)	Prec@1 76.562 (75.781)	Prec@5 91.406 (93.253)
Test: [20/391]	Time 0.813 (1.495)	Loss 0.9438 (1.0272)	Prec@1 78.125 (75.521)	Prec@5 95.312 (93.118)
Test: [30/391]	Time 0.587 (1.356)	Loss 0.9507 (1.0202)	Prec@1 74.219 (75.806)	Prec@5 94.531 (93.296)
Test: [40/391]	Time 0.940 (1.264)	Loss 1.2256 (0.9870)	Prec@1 75.781 (76.753)	Prec@5 91.406 (93.636)
Test: [50/391]	Time 0.612 (1.234)	Loss 1.2236 (1.0396)	Prec@1 73.438 (75.689)	Prec@5 90.625 (93.137)
Test: [60/391]	Time 0.839 (1.205)	Loss 1.4375 (1.0847)	Prec@1 64.062 (74.590)	Prec@5 88.281 (92.636)
Test: [70/391]	Time 0.723 (1.208)	Loss 1.0352 (1.0779)	Prec@1 77.344 (74.747)	Prec@5 92.188 (92.606)
Test: [80/391]	Time 0.640 (1.169)	Loss 1.0293 (1.0525)	Prec@1 78.906 (75.347)	Prec@5 92.969 (92.834)
Test: [90/391]	Time 0.759 (1.167)	Loss 0.9678 (1.0403)	Prec@1 75.781 (75.592)	Prec@5 94.531 (93.012)
Test: [100/391]	Time 0.699 (1.153)	Loss 0.8857 (1.0190)	Prec@1 79.688 (76.006)	Prec@5 94.53

### Test TTA with size*1.14

In [42]:
min_scale = 0.5
trn_tfms = [
        transforms.Resize(int(target_size*1.14)),
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

val_tfms = [transforms.Resize(int(target_size*1.14)), transforms.CenterCrop(target_size)] + tensor_tfm
val_dataset = datasets.ImageFolder(valdir,  transforms.Compose(val_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)
tta_rs_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

Test: [10/391]	Time 0.543 (1.979)	Loss 1.2217 (1.0012)	Prec@1 75.781 (75.781)	Prec@5 89.844 (93.253)
Test: [20/391]	Time 0.773 (1.638)	Loss 0.9014 (1.0233)	Prec@1 79.688 (75.781)	Prec@5 95.312 (93.155)
Test: [30/391]	Time 0.607 (1.564)	Loss 1.0059 (1.0198)	Prec@1 75.000 (75.706)	Prec@5 93.750 (93.196)
Test: [40/391]	Time 0.675 (1.439)	Loss 1.2031 (0.9880)	Prec@1 72.656 (76.543)	Prec@5 91.406 (93.617)
Test: [50/391]	Time 0.521 (1.413)	Loss 1.2422 (1.0473)	Prec@1 73.438 (75.398)	Prec@5 91.406 (93.015)
Test: [60/391]	Time 0.616 (1.350)	Loss 1.3984 (1.0915)	Prec@1 65.625 (74.488)	Prec@5 89.844 (92.533)
Test: [70/391]	Time 4.040 (1.351)	Loss 1.0420 (1.0838)	Prec@1 76.562 (74.681)	Prec@5 92.188 (92.562)
Test: [80/391]	Time 0.716 (1.304)	Loss 1.0352 (1.0608)	Prec@1 77.344 (75.183)	Prec@5 92.969 (92.814)
Test: [90/391]	Time 0.605 (1.276)	Loss 0.9790 (1.0476)	Prec@1 75.781 (75.489)	Prec@5 95.312 (92.986)
Test: [100/391]	Time 0.662 (1.293)	Loss 0.8789 (1.0282)	Prec@1 78.906 (75.882)	Prec@5 94.53

In [45]:
d = {
    'AR Mean': ar_batch_means[:-1],
    'DawnValidation': orig_prec5, 
    'TTA': tta_prec5, 
    'ARValidation': ar_rs_prec5, 
#     'Difference': np.array(ar_rs_prec5)-np.array(orig_prec5)
}
df = pd.DataFrame(data=d);
df.to_csv('ar_tests_tta.csv')
df

,AR Mean,DawnValidation,TTA,ARValidation
0,0.636959,88.28125,91.40625,92.18750
1,0.667781,95.31250,95.31250,95.31250
2,0.717153,94.53125,94.53125,92.96875
3,0.750000,89.06250,91.40625,92.18750
4,0.750000,91.40625,90.62500,91.40625
5,0.750000,88.28125,88.28125,89.84375
6,0.750127,92.18750,92.18750,92.18750
7,0.777394,90.62500,92.96875,92.96875
8,0.830958,95.31250,94.53125,96.87500
9,0.922669,94.53125,94.53125,94.53125


### TTA with AR

In [16]:
min_scale = 0.5
trn_tfms = [
        transforms.RandomResizedCrop(target_size, scale=(min_scale, 1.0)),
        transforms.RandomHorizontalFlip(),
    ] + tensor_tfm
aug_dataset = datasets.ImageFolder(valdir, transforms.Compose(trn_tfms))

aug_loader = torch.utils.data.DataLoader(
    aug_dataset, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

val_ar_tfms = [transforms.Resize(int(target_size*1.14)), RectangularCropTfm(idx2ar, target_size)]
val_dataset_ar_rs = ValDataset(valdir, val_ar_tfms+tensor_tfm)
val_loader = torch.utils.data.DataLoader(
    val_dataset_ar_rs, batch_size=val_bs, shuffle=False,
    num_workers=workers, pin_memory=True, sampler=val_sampler_ar)

In [17]:
tta_ar_rs_prec5 = validate(val_loader, model, criterion, aug_loader=aug_loader, num_augmentations=4)

Test: [10/391]	Time 0.600 (3.145)	Loss 1.1875 (0.9703)	Prec@1 75.781 (76.278)	Prec@5 91.406 (94.460)
Test: [20/391]	Time 3.100 (2.190)	Loss 0.9180 (0.9912)	Prec@1 78.125 (76.228)	Prec@5 95.312 (94.085)
Test: [30/391]	Time 0.538 (1.921)	Loss 0.9868 (0.9960)	Prec@1 75.000 (76.411)	Prec@5 92.188 (93.826)
Test: [40/391]	Time 0.696 (1.613)	Loss 1.2012 (0.9661)	Prec@1 73.438 (77.115)	Prec@5 91.406 (94.055)
Test: [50/391]	Time 0.632 (1.436)	Loss 1.1846 (1.0168)	Prec@1 74.219 (76.180)	Prec@5 90.625 (93.413)
Test: [60/391]	Time 2.159 (1.408)	Loss 1.3584 (1.0595)	Prec@1 66.406 (75.192)	Prec@5 89.844 (92.866)
Test: [70/391]	Time 0.551 (1.368)	Loss 1.0029 (1.0548)	Prec@1 78.125 (75.330)	Prec@5 92.188 (92.826)
Test: [80/391]	Time 2.978 (1.358)	Loss 0.9819 (1.0313)	Prec@1 77.344 (75.781)	Prec@5 93.750 (93.046)
Test: [90/391]	Time 0.535 (1.328)	Loss 0.8921 (1.0211)	Prec@1 78.906 (76.030)	Prec@5 96.875 (93.252)
Test: [100/391]	Time 0.567 (1.318)	Loss 0.8701 (1.0010)	Prec@1 80.469 (76.408)	Prec@5 94.53

In [18]:
df = pd.read_csv('ar_tests_tta.csv', index_col=0); df

,AR Mean,DawnValidation,TTA,ARValidation
0,0.636959,88.28125,91.40625,92.18750
1,0.667781,95.31250,95.31250,95.31250
2,0.717153,94.53125,94.53125,92.96875
3,0.750000,89.06250,91.40625,92.18750
4,0.750000,91.40625,90.62500,91.40625
5,0.750000,88.28125,88.28125,89.84375
6,0.750127,92.18750,92.18750,92.18750
7,0.777394,90.62500,92.96875,92.96875
8,0.830958,95.31250,94.53125,96.87500
9,0.922669,94.53125,94.53125,94.53125


In [19]:
df_2 = pd.DataFrame({'TTA_AR': tta_ar_rs_prec5})

In [21]:
df.to_csv('ar_tests_tta.csv')

In [20]:
df.join(df_2)

,AR Mean,DawnValidation,TTA,ARValidation,TTA_AR
0,0.636959,88.28125,91.40625,92.18750,91.40625
1,0.667781,95.31250,95.31250,95.31250,95.31250
2,0.717153,94.53125,94.53125,92.96875,92.18750
3,0.750000,89.06250,91.40625,92.18750,91.40625
4,0.750000,91.40625,90.62500,91.40625,90.62500
5,0.750000,88.28125,88.28125,89.84375,89.84375
6,0.750127,92.18750,92.18750,92.18750,92.18750
7,0.777394,90.62500,92.96875,92.96875,93.75000
8,0.830958,95.31250,94.53125,96.87500,96.87500
9,0.922669,94.53125,94.53125,94.53125,94.53125


In [ ]:
dat1 == pd pd..DataFrameDataFrame({({'dat1''dat1'::  [[99,,55]})]})
dat2 == pd pd..DataFrameDataFrame({({'dat2''dat2'::  [[77,,66]})]})
 >> dat1 dat1..joinjoin((dat2dat2))